# Detecção de Raças de Cachorros utilizando Redes Neurais

## Importações

### Importando biblioteca

In [157]:

from sys import platform
import os
import pandas as pd
from PIL import Image
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

### Importando Dataset

In [158]:
if platform == "linux" or platform == "linux2":
    folder_path = os.getcwd() + "/" + "low-resolution"
elif platform == "win32":
   folder_path = os.getcwd() + "\\" + "low-resolution"


list_images = []

for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)
    
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            
            file_path = os.path.join(subfolder_path, file)
            image_about = {}
            breed = subfolder.split("-")[-1]
            image_about["breed"] = breed
            image_about["filename"] = file
            image_about["path_file"] = file_path
            list_images.append(image_about)

df_images = pd.DataFrame.from_records(list_images)

### Pré-processamento

In [159]:
if platform == "linux" or platform == "linux2":
    os_path = "/" 
elif platform == "win32":
   os_path = "\\"

folder_path = os.getcwd() + os_path + "low-resolution"
slot_size = 20

#Retorna uma tupla com os path de cada imagem e o target randomizados
def load_path_of_dataset():
    arquivos = os.listdir(folder_path)
    subpastas = [subpasta for subpasta in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, subpasta))]
    imagens_path = []
    labels = []
    for subpasta in subpastas:
        subpasta_path = os.path.join(folder_path, subpasta)
        imagens = os.listdir(subpasta_path)
        imagens_path.extend([subpasta + os_path + s for s in imagens])
        label = subpasta.split("-")[2]
        labels.extend([label for s in imagens])
    # minha_lista = list(zip(imagens_path, labels))
    # random.shuffle(minha_lista)
    return imagens_path, labels

#função responsavel em realizar a divisão em slots
#recebe como parametro a tupla da função load_path_of_dataset, e o numero da interação para carregar o dataframe
#retorna uma tupla menor, e o numero máximo de interações posiveis até o fim do dataset
def slot_loading(interacao, imagens_amostradas, imagens_labels):
    if interacao <= 0:
        raise ValueError('O valor da interação, deve ser maior que zero ! ')
    elif interacao > (interacao * len(imagens_amostradas)):
        raise ValueError('O valor da interação, é muito grande !')
    
    start_index = (interacao - 1) * slot_size
    end_index = interacao * slot_size
    if end_index > len(imagens_amostradas):
        return imagens_amostradas[start_index:], imagens_labels[start_index:]
    
    return imagens_amostradas[start_index:end_index], tf.convert_to_tensor(imagens_labels[start_index:end_index]), len(imagens_amostradas) // slot_size

#Recebe como parametro uma tupla
#retorna um dataset já pré-processado e formatados
def get_images(imagens_amostradas):
    img_size_height = 300
    img_size_width = 400
    images = []
    for imagem_nome in imagens_amostradas:
        try:
            imagem_path = os.path.join(folder_path, imagem_nome)
            imagem = Image.open(imagem_path)
            imagem = imagem.resize((img_size_width, img_size_height))  # Redimensiona para um tamanho comum
            imagem_array = np.array(imagem) / 255.0  # Normaliza dividindo por 255
            images.append(imagem_array)
        except Exception as error:
            print("ocorreu um erro com imagem_path: ", imagem_path, " imagem_nome " + imagem_nome," imagem_nome[0] " + imagem_nome[0], " imagens_amostradas: " , imagens_amostradas)
    images = tf.stack(images)
    return images


#### Exemplo de utilização

In [160]:
X, y = load_path_of_dataset()
x_new, y_new, interations = slot_loading(1, X, y)
# slot_path_dataset, interations = slot_loading(1,path_dataset)
# dataset = get_images(x_new);
# print(dataset)

## Separação do Conjunto de teste e treino

In [161]:
# Sua tupla
X, y = load_path_of_dataset()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = y.tolist()
# Codifique os rótulos como vetores one-hot
y = tf.one_hot(y, depth=133)
# Converta o tensor para um array numpy
y = y.numpy()
# Separando 70% para treino e 30% para teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

# Exibindo as dimensões dos conjuntos de treino e teste
print("Dimensões do conjunto de treino:", len(X_treino), len(y_treino))
print("Dimensões do conjunto de teste:", len(X_teste), len(y_teste))

Dimensões do conjunto de treino: 49302 49302
Dimensões do conjunto de teste: 21130 21130


## Criação do modelo

In [162]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
# Model Architecture
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(300,400,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Conv2D(filters=128 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(133,activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_68 (Conv2D)          (None, 300, 400, 16)      208       
                                                                 
 max_pooling2d_68 (MaxPooli  (None, 150, 200, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_69 (Conv2D)          (None, 150, 200, 32)      2080      
                                                                 
 max_pooling2d_69 (MaxPooli  (None, 75, 100, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_70 (Conv2D)          (None, 75, 100, 64)       8256      
                                                                 
 max_pooling2d_70 (MaxPooli  (None, 37, 50, 64)      

#### Treinamento em batches 

In [163]:

# X = [item[0] for item in path_dataset]  # Recursos (features) da tupla
# y = [item[1] for item in path_dataset]   # Rótulos (labels) da tupla


# Treinar o modelo
x_new, y_new, i = slot_loading(1, X_treino, y_treino)
x_new_tensor = get_images(x_new)

model.fit(x_new_tensor, y_new)
for o in range(1, i):
    print( o," of " ,i)
    x_new, y_new = None, None
    x_new, y_new, i = slot_loading(o, X_treino, y_treino)
    x_new = get_images(x_new)
    model.fit(x_new , y_new)
# # Avaliar o modelo
# score = model.score(dados_teste, rotulos_teste)
# print("Acurácia do modelo:", score)




['274-n000110-Shetland_sheepdog/n136341.jpeg', '806-n000129-papillon/n170528.jpg', '3336-n000121-chinese_rural_dog/n147394.jpeg', '2909-n000116-Cardigan/n139660.jpeg', '2192-n000088-Samoyed/n125726.jpeg', '3336-n000121-chinese_rural_dog/n149939.jpeg', '3336-n000121-chinese_rural_dog/n149362.jpeg', '2192-n000088-Samoyed/n124595.jpg', '215-n000075-Chesapeake_Bay_retriever/n121448.jpg', '3580-n000122-Labrador_retriever/n151980.jpg', '207-n000011-Irish_setter/n107660.jpg', '253-n000107-Japanese_Spitzes/n133079.jpeg', '1043-n000001-Shiba_Dog/n100255.jpg', '5355-n000126-golden_retriever/n160225.jpeg', '206-n000051-Saluki/n114850.jpg', '221-n000055-EntleBucher/n117419.jpg', '2342-n000102-miniature_schnauzer/n129881.jpg', '276-n000112-English_springer/n136751.jpg', '230-n000084-Boston_bull/n123835.jpg', '211-n000059-German_short_haired_pointer/n118379.jpg']
1/1 [==============================] - 2s 2s/step - loss: 0.6849 - accuracy: 0.0000e+00
1  of  2465
1/1 [==============================] -